In [1]:
pip install openai-agents

  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
   ---------------------------------------- 0.0/720.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/720.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/720.4 kB ? eta -:--:--
   -------------- ------------------------- 262.1/720.4 kB ? eta -:--:--
   -------------- ------------------------- 262.1/720.4 kB ? eta -:--:--
   -------------- ------------------------- 262.1/720.4 kB ? eta -:--:--
   -------------------------------------- 720.4/720.4 kB 733.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------- ----------------------- 0.8/2.0 MB 4.2 MB/s eta 0:00:01
   -------------------

In [ ]:
from litellm import completion
import os

api_key ="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"
messages=[
       {"role": "user", "content": "Hello how are you ?"}
   ]
model = "groq/qwen-qwq-32b"
test = completion(messages = messages, model = model, api_key=api_key)

test.choices[0].message.content

In [8]:
test.choices[0].message.content

ModelResponse(id='chatcmpl-11de434a-ca34-43d8-912e-07e901c82837', created=1747989353, model='qwen-qwq-32b', object='chat.completion', system_fingerprint='fp_1e88ca32eb', choices=[Choices(finish_reason='stop', index=0, message=Message(content='\n<think>\nOkay, the user said "Hello how are you?" I need to respond appropriately. Let\'s see, first I should greet them back. "Hello!" That\'s friendly. Then, answer their question. How am I? Since I\'m an AI, I don\'t have feelings, but I should say I\'m functioning well. Maybe add a微笑表情 to keep it warm. Then, ask them back how they are doing. Oh, the user might want a conversation, so I should invite them to share what\'s on their mind. Keep it open-ended so they feel comfortable to ask for help. Let me put that together naturally.\n</think>\n\nHello! I\'m just a computer program, so I don\'t have feelings, but I\'m here and ready to help you! How can I assist you today? 😊', role='assistant', tool_calls=None, function_call=None, provider_spec

In [9]:
test.choices[0].message.content

'\n<think>\nOkay, the user said "Hello how are you?" I need to respond appropriately. Let\'s see, first I should greet them back. "Hello!" That\'s friendly. Then, answer their question. How am I? Since I\'m an AI, I don\'t have feelings, but I should say I\'m functioning well. Maybe add a微笑表情 to keep it warm. Then, ask them back how they are doing. Oh, the user might want a conversation, so I should invite them to share what\'s on their mind. Keep it open-ended so they feel comfortable to ask for help. Let me put that together naturally.\n</think>\n\nHello! I\'m just a computer program, so I don\'t have feelings, but I\'m here and ready to help you! How can I assist you today? 😊'

In [10]:
import re

def clean_groq_output(raw_text: str) -> str:
    # 1. Remove <think>...</think> blocks
    cleaned = re.sub(r'<think>.*?</think>', '', raw_text, flags=re.DOTALL)
    
    # 2. Replace escape sequences with normal formatting
    cleaned = cleaned.replace('\\n', '\n')  # Convert literal '\n' to real newlines
    cleaned = cleaned.replace('\\\'', "'")  # Fix escaped single quotes
    cleaned = cleaned.replace('\\"', '"')   # Fix escaped double quotes
    cleaned = cleaned.replace('\\\\', '\\') # Fix double backslashes

    # 3. Remove any leading/trailing whitespace
    cleaned = cleaned.strip()

    return cleaned

In [12]:
clean_output = clean_groq_output(test.choices[0].message.content)
print(clean_output)


Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today? 😊


In [21]:
from agents import Agent, Runner, function_tool, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
from agents import set_tracing_disabled

set_tracing_disabled(True)
# This is how we define a tool
# The agent will have acces to arg type and function docstring to understand how he can use the tool
@function_tool
def get_weather(city: str):
    """Get the weathe in a specific citry

    Args:
      city(str): The city name where we want to get the weather

    Retun:
      str: A string that indicate the weather in the city

    """
    print(f"[debug] getting weather for {city}")
    return f"The weather in {city} is sunny."


agent = Agent(
    name="Assistant",
    instructions="You give information about meteo",
    model=LitellmModel(model=model, api_key=api_key),
    tools=[get_weather],
)

result = await Runner.run(agent, "What's the weather in Tokyo?")
print(result.final_output)

[debug] getting weather for Tokyo
The current weather in Tokyo is sunny.


In [36]:
import re
import json
import os
from litellm import completion
from agents import Agent, Runner, function_tool, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel

# ——————— Utilities ———————

def clean_groq_output(raw_text: str) -> str:
    cleaned = re.sub(r'<think>.*?</think>', '', raw_text, flags=re.DOTALL)
    cleaned = (cleaned
        .replace('\\n', '\n')
        .replace("\\'", "'")
        .replace('\\"', '"')
        .replace('\\\\', '\\')
    )
    return cleaned.strip()

API_KEY = os.getenv("GROQ_API_KEY", "<your-api-key-here>")
MODEL   = "groq/qwen-qwq-32b"

# ——————— Tools ———————

@function_tool
def retrieve_evidence(query: str, documents: list) -> list:
    """
    Stub RAG: returns the documents list (or [] if none).
    """
    return documents or []

@function_tool
def analyze_greenwashing(marketing_statement: str, evidence: list) -> str:
    """
    Returns a JSON string:
      {
        "is_greenwashing": bool,
        "explanation": str
      }
    """
    if not evidence:
        result = {
            "is_greenwashing": False,
            "explanation": "No company information available; cannot confirm or deny greenwashing."
        }
        return json.dumps(result)
    
    prompt = f"""
You are a sustainability analyst.
User claim: "{marketing_statement}"
Company info (RAG results): {json.dumps(evidence, indent=2)}

Respond with valid JSON exactly in this form:
{{ 
  "is_greenwashing": <true|false>,
  "explanation": "<quote relevant passages and justify your decision>"
}}
"""
    resp = completion(
        messages=[
            {"role": "system", "content": "Detect greenwashing in marketing copy."},
            {"role": "user",   "content": prompt}
        ],
        model=MODEL,
        api_key=API_KEY
    )
    raw = resp.choices[0].message.content
    return clean_groq_output(raw)

@function_tool
def rewrite_statement(original_statement: str, analysis_json: str) -> str:
    """
    Args:
      analysis_json: the *string* returned by analyze_greenwashing
    Returns a JSON string:
      {
        "revised_statement": str,
        "justification": str
      }
    """
    analysis = json.loads(analysis_json)
    if not analysis.get("is_greenwashing", False):
        # echo original + explanation
        return json.dumps({
            "revised_statement": original_statement,
            "justification": analysis["explanation"]
        })
    
    prompt = f"""
You are a copy editor focused on sustainability accuracy.
Original statement: "{original_statement}"
Greenwashing analysis:
{json.dumps(analysis, indent=2)}

Respond with valid JSON exactly in this form:
{{
  "revised_statement": "<rewrite to avoid any misleading claims>",
  "justification": "<explain how the rewrite improves honesty>"
}}
"""
    resp = completion(
        messages=[
            {"role": "system", "content": "Rewrite marketing copy to avoid greenwashing."},
            {"role": "user",   "content": prompt}
        ],
        model=MODEL,
        api_key=API_KEY
    )
    raw = resp.choices[0].message.content
    return clean_groq_output(raw)

# ——————— Agent setup ———————

set_tracing_disabled(True)
agent = Agent(
    name="GreenwashingInspector",
    instructions=(
        "You are an AI that validates marketing claims for greenwashing. "
        "Given user docs, retrieve evidence, analyze the claim, "
        "and rewrite it honestly if needed."
    ),
    model=LitellmModel(model=MODEL, api_key=API_KEY),
    tools=[retrieve_evidence, analyze_greenwashing, rewrite_statement],
)

# ——————— Example run ———————

async def main():
    statement = "Our operations are 100% carbon neutral"
    # pass an empty list so retrieve_evidence → []
    result = await Runner.run(agent, statement, documents=[])
    
    # result.final_output will be the agent’s last message,
    # which, because of how we set instructions, should be the JSON
    print("Agent says:", result.final_output)

# If you want to run it directly:
# import asyncio; asyncio.run(main())


if __name__ == "__main__":
    import asyncio
    # asyncio.run(main())
    await main()
    
    #use await instead of async in ipynb 


TypeError: run() got an unexpected keyword argument 'documents'

In [49]:
import re
import json
import os
from litellm import completion
from agents import Agent, Runner, function_tool, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel

# ——————— Utilities ———————

def clean_groq_output(raw_text: str) -> str:
    cleaned = re.sub(r'<think>.*?</think>', '', raw_text, flags=re.DOTALL)
    cleaned = (
        cleaned
        .replace('\\n', '\n')
        .replace("\\'", "'")
        .replace('\\"', '"')
        .replace('\\\\', '\\')
    )
    return cleaned.strip()

API_KEY = "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"
MODEL   = "groq/qwen-qwq-32b"

# ——————— Tools ———————

@function_tool
def retrieve_evidence(query: str) -> list:
    """
    Stub RAG: always returns an empty list (no company docs).
    """
    return ["Our company has almost reached a carbon netural status with an 80% reduction in carbon emissions"]

@function_tool
def analyze_greenwashing(marketing_statement: str, evidence: list) -> str:
    """
    Returns a JSON string:
      {
        "is_greenwashing": bool,
        "explanation": str
      }
    """
    if not evidence:
        return json.dumps({
            "is_greenwashing": False,
            "explanation": "No company information available; cannot confirm or deny greenwashing."
        })

    prompt = f"""
You are a sustainability analyst.
User claim: "{marketing_statement}"
Company info (RAG results): {json.dumps(evidence, indent=2)}

Respond with valid JSON exactly in this form:
{{ 
  "is_greenwashing": <true|false>,
  "explanation": "<quote relevant passages and justify your decision>"
}}
"""
    resp = completion(
        messages=[
            {"role": "system", "content": "Detect greenwashing in marketing copy."},
            {"role": "user",   "content": prompt}
        ],
        model=MODEL,
        api_key=API_KEY
    )
    return clean_groq_output(resp.choices[0].message.content)

@function_tool
def rewrite_statement(original_statement: str, analysis_json: str) -> str:
    """
    Returns a JSON string:
      {
        "revised_statement": str,
        "justification": str
      }
    """
    analysis = json.loads(analysis_json)
    if not analysis.get("is_greenwashing", False):
        return json.dumps({
            "revised_statement": original_statement,
            "justification": analysis["explanation"]
        })

    prompt = f"""
You are a copy editor focused on sustainability accuracy.
Original statement: "{original_statement}"
Greenwashing analysis:
{json.dumps(analysis, indent=2)}

Respond with valid JSON exactly in this form:
{{
  "revised_statement": "<rewrite to avoid any misleading claims>",
  "justification": "<explain how the rewrite improves honesty>"
}}
"""
    resp = completion(
        messages=[
            {"role": "system", "content": "Rewrite marketing copy to avoid greenwashing."},
            {"role": "user",   "content": prompt}
        ],
        model=MODEL,
        api_key=API_KEY
    )
    return clean_groq_output(resp.choices[0].message.content)

# ——————— Agent setup ———————

set_tracing_disabled(True)
agent = Agent(
    name="GreenwashingInspector",
    instructions=(
        "You are an AI that validates marketing claims for greenwashing. "
        "First retrieve any evidence, then analyze the claim, and finally rewrite it honestly if needed."
    ),
    model=LitellmModel(model=MODEL, api_key=API_KEY),
    tools=[retrieve_evidence, analyze_greenwashing, rewrite_statement],
)

# ——————— Example run ———————

async def main():
    statement = "Our operations are 100% carbon neutral"
    result = await Runner.run(agent, statement)
    print(result.final_output)

if __name__ == "__main__":
    import asyncio
    #asyncio.run(main())
    await main()


The original claim "Our operations are 100% carbon neutral" is likely greenwashing due to lack of verifiable evidence. The suggested honest revision is:  

**Revised Statement:**  
*"We have reduced our carbon emissions by 80% through verified sustainability initiatives and continue to measure progress toward our net-zero goals."*  

**Why this works:**  
1. Anchors the claim in measurable progress (80% reduction) instead of an absolute unverified goal.  
2. Mentions ongoing verification and measurement processes to build credibility.  
3. Avoids absolute terms like "100% carbon neutral" without third-party certification or transparent reporting.  

Always pair such statements with publicly accessible emissions reports or certifications (e.g., Scope 1-3 emission data, SDG alignment) to reinforce trust.


In [61]:
import re
import json
import os
from litellm import completion
from agents import Agent, Runner, function_tool, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel

# ——————— Utilities ———————

def clean_groq_output(raw_text: str) -> str:
    """
    Cleans raw model output by:
      1. Removing any internal <think>...</think> reasoning blocks
      2. Un-escaping literal sequences (\n, \", \', \\\\)
      3. Stripping leading/trailing whitespace

    Input:
      raw_text (str): The raw string output from the LLM
    Output:
      str: Cleaned text, ready for JSON parsing or direct use
    """
    cleaned = re.sub(r'<think>.*?</think>', '', raw_text, flags=re.DOTALL)
    cleaned = (
        cleaned
        .replace('\\n', '\n')
        .replace("\\'", "'")
        .replace('\\"', '"')
        .replace('\\\\', '\\')
    )
    return cleaned.strip()

API_KEY = "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"
MODEL   = "groq/qwen-qwq-32b"

def parse_agent_output(output: str) -> dict:
    """
    Parse the agent's six-section response into a dict with keys:
      - retrieved_evidence
      - conclusion
      - justification
      - rewritten_statement
      - suggestions_for_improvement
      - greenwashing_category
    """
    pattern = (
        r"1\.\s*Retrieved Evidence\s*(.*?)\s*"
        r"2\.\s*Conclusion\s*(.*?)\s*"
        r"3\.\s*Justification\s*(.*?)\s*"
        r"4\.\s*Rewritten Statement\s*(.*?)\s*"
        r"5\.\s*Suggestions for Improvement\s*(.*?)\s*"
        r"6\.\s*Greenwashing Category\s*(.*)"
    )
    match = re.search(pattern, output, re.DOTALL)
    if not match:
        raise ValueError("Response did not match expected format")
    return {
        "retrieved_evidence": match.group(1).strip(),
        "conclusion": match.group(2).strip(),
        "justification": match.group(3).strip(),
        "rewritten_statement": match.group(4).strip(),
        "suggestions_for_improvement": match.group(5).strip(),
        "greenwashing_category": match.group(6).strip(),
    }

# ——————— Tools ———————

@function_tool
def retrieve_evidence(query: str) -> list:
    """
    Stub RAG tool that retrieves company evidence snippets for a given marketing claim.

    Input:
      query (str): The marketing statement or claim to validate.
    Output:
      list of str: A list of evidence snippets (company report lines). Empty if none.

  
    """
    return ["Our company has almost reached a carbon neutral status with an 80% reduction in carbon emissions"]

@function_tool
def analyze_greenwashing(marketing_statement: str, evidence: list) -> str:
    """
    Analyzes whether the marketing statement is greenwashing based only on the evidence provided in the rag results output.

    Input:
      marketing_statement (str): The claim to analyze.
      evidence (list of str): Extracted company information snippets.
    Output:
      str: A JSON-formatted string with fields:
        {
          "is_greenwashing": bool,
          "explanation": str  # Justification quoting evidence if any
        }

    Behavior:
      - If `evidence` is empty, returns is_greenwashing=false and a fallback explanation.
      - Otherwise, prompts the LLM to detect misleading or unsupported claims.
    """
    if not evidence:
        return json.dumps({
            "is_greenwashing": False,
            "explanation": "No company information available; cannot confirm or deny greenwashing."
        })

    prompt = f"""
You are a sustainability analyst.
User claim: "{marketing_statement}"
Company info (RAG results): {json.dumps(evidence, indent=2)}

Respond with valid JSON exactly in this form:
{{ 
  "is_greenwashing": <true|false>,
  "explanation": "<quote relevant passages and justify your decision>"
}}
"""
    resp = completion(
        messages=[
            {"role": "system", "content": "Detect greenwashing in marketing copy."},
            {"role": "user",   "content": prompt}
        ],
        model=MODEL,
        api_key=API_KEY
    )
    return clean_groq_output(resp.choices[0].message.content)

@function_tool
def rewrite_statement(original_statement: str, analysis_json: str) -> str:
    """
    Rewrites a marketing statement to remove any greenwashing, based on analysis.

    Input:
      original_statement (str): The original marketing copy.
      analysis_json (str): JSON output from `analyze_greenwashing` tool.
    Output:
      str: A JSON-formatted string with fields:
        {
          "revised_statement": str,  # Corrected marketing copy
          "justification": str       # How the rewrite improves honesty
        }

    Behavior:
      - If analysis_json indicates no greenwashing, echoes original with explanation.
      - Otherwise, prompts the LLM to produce a rewritten, accurate statement.
    """
    analysis = json.loads(analysis_json)
    if not analysis.get("is_greenwashing", False):
        return json.dumps({
            "revised_statement": original_statement,
            "justification": analysis["explanation"]
        })

    prompt = f"""
You are a copy editor focused on sustainability accuracy.
Original statement: "{original_statement}"
Greenwashing analysis:
{json.dumps(analysis, indent=2)}

Respond with valid JSON exactly in this form:
{{
  "revised_statement": "<rewrite to avoid any misleading claims>",
  "justification": "<explain how the rewrite improves honesty>"
}}
"""
    resp = completion(
        messages=[
            {"role": "system", "content": "Rewrite marketing copy to avoid greenwashing."},
            {"role": "user",   "content": prompt}
        ],
        model=MODEL,
        api_key=API_KEY
    )
    return clean_groq_output(resp.choices[0].message.content)

# ——————— Agent setup ———————

set_tracing_disabled(True)
agent = Agent(
    name="GreenwashingInspector",
    instructions=(
      "You are an AI that detects and corrects greenwashing in marketing statements.\n\n"
      "Use these tools:\n"
      "- retrieve_evidence(query) to get only factual company sustainability data.\n"
      "- analyze_greenwashing(statement, evidence) to decide if the claim is greenwashing.\n"
      "- rewrite_statement(original, analysis_json) to output a corrected version.\n\n"
      "For every input statement, return exactly:\n\n"
      "1. Retrieved Evidence\n"
      "   Concise facts from retrieve_evidence(query).\n\n"
      "2. Conclusion\n"
      "   **Greenwashing Detected** or **No Greenwashing Detected**.\n\n"
      "3. Justification\n"
      "   Brief reason quoting only the retrieved evidence.\n\n"
      "4. Rewritten Statement\n"
      "   Accurate, transparent claim from rewrite_statement.\n\n"
      "5. Suggestions for Improvement\n"
      "   2–3 concise, evidence-based edits.\n\n"
      "6. Greenwashing Category\n"
      "   Choose one and explain with evidence:\n"
      "     a) Ambiguous Claims: Vague, metric-free language.\n"
      "     b) Hidden Trade-offs: Only one eco aspect highlighted.\n"
      "     c) Visual Deception: Misleading imagery.\n"
      "     d) Misleading Metrics: Incorrect or unverified numbers.\n"
      "     e) Empty Promises: Commitments without a real plan.\n"
      "     f) Unverified Certifications: Fake or non-verified labels.\n"
      "     g) Compliance as Leadership: Framing legal duty as a choice.\n"

    ),
    model=LitellmModel(model=MODEL, api_key=API_KEY),
    tools=[retrieve_evidence, analyze_greenwashing, rewrite_statement],
)

# ——————— Example run ———————

async def main():
    # Test with a sample marketing statement
    statement = "Our operations are 100% carbon neutral"
    result = await Runner.run(agent, statement)
    print(parse_agent_output(result.final_output))

if __name__ == "__main__":
    import asyncio
    #asyncio.run(main())
    await main()

{'retrieved_evidence': '"Our company has almost reached a carbon neutral status with an 80% reduction in carbon emissions"', 'conclusion': '**Greenwashing Detected**', 'justification': 'The claim of "100% carbon neutral" contradicts the evidence showing only an 80% reduction in emissions, with no mention of offsets orverified certifications to bridge the remaining 20%.', 'rewritten_statement': '"We’ve reduced our operational carbon emissions by 80% and are actively working toward full carbon neutrality by 2030 through renewable energy investments and certified carbon offset projects."', 'suggestions_for_improvement': '- Replace "100% carbon neutral" with the verified 80% reduction figure  \n   - Specify actions (e.g., renewable energy adoption) and timelines for achieving neutrality  \n   - Reference third-party certifications (if applicable) or verified offset programs', 'greenwashing_category': '**d) Misleading Metrics**: The claim uses an absolute percentage ("100%") without evidenc

In [64]:
import re
import json
import os
from litellm import completion
from agents import Agent, Runner, function_tool, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel

# ——————— Utilities ———————

def clean_groq_output(raw_text: str) -> str:
    """
    Cleans raw model output by:
      1. Removing any internal <think>...</think> reasoning blocks
      2. Un-escaping literal sequences (\n, \", \', \\\\)
      3. Stripping leading/trailing whitespace

    Input:
      raw_text (str): The raw string output from the LLM
    Output:
      str: Cleaned text, ready for JSON parsing or direct use
    """
    cleaned = re.sub(r'<think>.*?</think>', '', raw_text, flags=re.DOTALL)
    cleaned = (
        cleaned
        .replace('\\n', '\n')
        .replace("\\'", "'")
        .replace('\\"', '"')
        .replace('\\\\', '\\')
    )
    return cleaned.strip()

API_KEY = "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"
MODEL   = "groq/qwen-qwq-32b"

def parse_agent_output(output: str) -> dict:
    """
    Parse the agent's six-section response into a dict with keys:
      - retrieved_evidence
      - conclusion
      - justification
      - rewritten_statement
      - suggestions_for_improvement
      - greenwashing_category
    """
    pattern = (
        r"1\.\s*Retrieved Evidence\s*(.*?)\s*"
        r"2\.\s*Conclusion\s*(.*?)\s*"
        r"3\.\s*Justification\s*(.*?)\s*"
        r"4\.\s*Rewritten Statement\s*(.*?)\s*"
        r"5\.\s*Suggestions for Improvement\s*(.*?)\s*"
        r"6\.\s*Greenwashing Category\s*(.*)"
    )
    match = re.search(pattern, output, re.DOTALL)
    if not match:
        raise ValueError("Response did not match expected format")
    return {
        "retrieved_evidence": match.group(1).strip(),
        "conclusion": match.group(2).strip(),
        "justification": match.group(3).strip(),
        "rewritten_statement": match.group(4).strip(),
        "suggestions_for_improvement": match.group(5).strip(),
        "greenwashing_category": match.group(6).strip(),
    }

# ——————— Tools ———————

@function_tool
def retrieve_evidence(query: str) -> list:
    """
    Stub RAG tool that retrieves company evidence snippets for a given marketing claim.

    Input:
      query (str): The marketing statement or claim to validate.
    Output:
      list of str: A list of evidence snippets (company report lines). Empty if none.

  
    """
    return ["Our company has almost reached a carbon neutral status with an 80% reduction in carbon emissions"]

@function_tool
def retrieve_laws(query: str) -> list:
    """
    Stub tool that retrieves relevant greenwashing laws or regulations for a given claim.
    """
    return [
        "EU Unfair Commercial Practices Directive prohibits misleading environmental claims.",
        "French AGEC law requires proof for any eco-label claim."
    ]

@function_tool
def analyze_greenwashing(marketing_statement: str, evidence: list) -> str:
    """
    Analyzes whether the marketing statement is greenwashing based only on the evidence provided in the rag results output.

    Input:
      marketing_statement (str): The claim to analyze.
      evidence (list of str): Extracted company information snippets.
    Output:
      str: A JSON-formatted string with fields:
        {
          "is_greenwashing": bool,
          "explanation": str  # Justification quoting evidence if any
        }

    Behavior:
      - If `evidence` is empty, returns is_greenwashing=false and a fallback explanation.
      - Otherwise, prompts the LLM to detect misleading or unsupported claims.
    """
    if not evidence:
        return json.dumps({
            "is_greenwashing": False,
            "explanation": "No company information available; cannot confirm or deny greenwashing."
        })

    prompt = f"""
You are a sustainability analyst.
User claim: "{marketing_statement}"
Company info (RAG results): {json.dumps(evidence, indent=2)}

Respond with valid JSON exactly in this form:
{{ 
  "is_greenwashing": <true|false>,
  "explanation": "<quote relevant passages and justify your decision>"
}}
"""
    resp = completion(
        messages=[
            {"role": "system", "content": "Detect greenwashing in marketing copy."},
            {"role": "user",   "content": prompt}
        ],
        model=MODEL,
        api_key=API_KEY
    )
    return clean_groq_output(resp.choices[0].message.content)

@function_tool
def rewrite_statement(original_statement: str, analysis_json: str) -> str:
    """
    Rewrites a marketing statement to remove any greenwashing, based on analysis.

    Input:
      original_statement (str): The original marketing copy.
      analysis_json (str): JSON output from `analyze_greenwashing` tool.
    Output:
      str: A JSON-formatted string with fields:
        {
          "revised_statement": str,  # Corrected marketing copy
          "justification": str       # How the rewrite improves honesty
        }

    Behavior:
      - If analysis_json indicates no greenwashing, echoes original with explanation.
      - Otherwise, prompts the LLM to produce a rewritten, accurate statement.
    """
    analysis = json.loads(analysis_json)
    if not analysis.get("is_greenwashing", False):
        return json.dumps({
            "revised_statement": original_statement,
            "justification": analysis["explanation"]
        })

    prompt = f"""
You are a copy editor focused on sustainability accuracy.
Original statement: "{original_statement}"
Greenwashing analysis:
{json.dumps(analysis, indent=2)}

Respond with valid JSON exactly in this form:
{{
  "revised_statement": "<rewrite to avoid any misleading claims>",
  "justification": "<explain how the rewrite improves honesty>"
}}
"""
    resp = completion(
        messages=[
            {"role": "system", "content": "Rewrite marketing copy to avoid greenwashing."},
            {"role": "user",   "content": prompt}
        ],
        model=MODEL,
        api_key=API_KEY
    )
    return clean_groq_output(resp.choices[0].message.content)

# ——————— Agent setup ———————

set_tracing_disabled(True)
agent = Agent(
    name="GreenwashingInspector",
    instructions=(
      "You are an AI that detects and corrects greenwashing in marketing statements.\n\n"
      "Use these tools:\n"
      "- retrieve_evidence(query) to get only factual company sustainability data.\n"
      "- retrieve_laws(query) to get factual laws and regulations related to greenwashing.\n"
      "- analyze_greenwashing(statement, evidence) to decide if the claim is greenwashing.\n"
      "- rewrite_statement(original, analysis_json) to output a corrected version.\n\n"
      "For every input statement, return exactly:\n\n"
      "1. Retrieved Evidence\n"
      "   Concise facts from retrieve_evidence(query).\n\n"
      "2. Conclusion\n"
      "   **Greenwashing Detected** or **No Greenwashing Detected**.\n\n"
      "3. Justification\n"
      "   Brief reason quoting only the retrieved evidence. Here also refer to the retrieved laws and regulations that are potentially violated\n\n"
      "4. Rewritten Statement\n"
      "   Accurate, transparent claim from rewrite_statement.\n\n"
      "5. Suggestions for Improvement\n"
      "   2–3 concise, evidence-based edits.\n\n"
      "6. Greenwashing Category\n"
      "   Choose one and explain with evidence:\n"
      "     a) Ambiguous Claims: Vague, metric-free language.\n"
      "     b) Hidden Trade-offs: Only one eco aspect highlighted.\n"
      "     c) Visual Deception: Misleading imagery.\n"
      "     d) Misleading Metrics: Incorrect or unverified numbers.\n"
      "     e) Empty Promises: Commitments without a real plan.\n"
      "     f) Unverified Certifications: Fake or non-verified labels.\n"
      "     g) Compliance as Leadership: Framing legal duty as a choice.\n"

    ),
    model=LitellmModel(model=MODEL, api_key=API_KEY),
    tools=[retrieve_evidence, retrieve_laws, analyze_greenwashing, rewrite_statement],
)

# ——————— Example run ———————

async def main():
    # Test with a sample marketing statement
    statement = "Our operations are 100% carbon neutral"
    result = await Runner.run(agent, statement)
    print(parse_agent_output(result.final_output))

if __name__ == "__main__":
    import asyncio
    #asyncio.run(main())
    await main()

{'retrieved_evidence': '- Company has only reduced emissions by 80%, not fully carbon neutral.', 'conclusion': '**Greenwashing Detected**', 'justification': 'The claim of "100% carbon neutral" contradicts evidence showing only 80% reduction in emissions (missing 20% of required mitigation). This misleads consumers under the EU Unfair Commercial Practices Directive, which prohibits false environmental claims.', 'rewritten_statement': '"Through our sustainability initiatives, we’ve reduced operational carbon emissions by 80% since 2020 and are actively pursuing additional offsets and reductions to achieve full carbon neutrality."', 'suggestions_for_improvement': '- Include third-party verification data.  \n- Specify how the remaining 20% will be addressed (e.g., investments in carbon removal projects).', 'greenwashing_category': '**d) Misleading Metrics**: The claim uses an exact percentage ("100%") that exceeds independently verified emissions reduction data (80% reduction), without cla